<a href="https://colab.research.google.com/github/thisishasan/prog_mod_b/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [417]:
!wget -nc 'https://raw.githubusercontent.com/thisishasan/prog_mod_b/refs/heads/main/data.csv'

File ‘data.csv’ already there; not retrieving.



In [406]:
import pandas as pd

df = pd.read_csv("data.csv")

df = df[['Principio Attivo', 'Descrizione Gruppo', 'Denominazione e Confezione', 'Titolare AIC', 'AIC', 'Codice Gruppo Equivalenza']]

df = df.iloc[472-2:804-2]

In [407]:
# Display the first 5 rows of the DataFrame
print(df.head())

    Principio Attivo                      Descrizione Gruppo  \
470    Amitriptilina  AMITRIPTILINA 25MG 25 UNITA' USO ORALE   
471    Amitriptilina  AMITRIPTILINA 25MG 30 UNITA' USO ORALE   
472    Amitriptilina    AMITRIPTILINA 40MG/ML 20ML USO ORALE   
473    Amitriptilina    AMITRIPTILINA 40MG/ML 20ML USO ORALE   
474    Amitriptilina    AMITRIPTILINA 40MG/ML 20ML USO ORALE   

                 Denominazione e Confezione                 Titolare AIC  \
470              TRIPTIZOL*25 cpr riv 25 mg  SIT LABORATORIO FARMAC. Srl   
471                ADEPRIL*30 cpr riv 25 mg                 TEOFARMA Srl   
472        LAROXYL*orale gtt 20 ml 40 mg/ml                 TEOFARMA Srl   
473  AMITRIPTILINA*orale gtt 20 ml 40 mg/ml                       EG SpA   
474  AMITRIPTILINA*orale gtt 20 ml 40 mg/ml             DOC GENERICI Srl   

          AIC Codice Gruppo Equivalenza  
470  19803016                       HOB  
471  20019028                       HOC  
472  19906054                   

In [408]:
# Display the last 5 rows of the DataFrame
print(df.tail())

    Principio Attivo                     Descrizione Gruppo  \
797     Aripiprazolo  ARIPIPRAZOLO 15MG 28 UNITA' USO ORALE   
798     Aripiprazolo  ARIPIPRAZOLO 15MG 28 UNITA' USO ORALE   
799     Aripiprazolo  ARIPIPRAZOLO 15MG 28 UNITA' USO ORALE   
800     Aripiprazolo  ARIPIPRAZOLO 15MG 28 UNITA' USO ORALE   
801     Aripiprazolo  ARIPIPRAZOLO 15MG 28 UNITA' USO ORALE   

      Denominazione e Confezione                   Titolare AIC       AIC  \
797    ARIPIPRAZOLO*28 cpr 15 mg                         EG SpA  43801101   
798    ARIPIPRAZOLO*28 cpr 15 mg                   ZENTIVA K.S.  44265128   
799    ARIPIPRAZOLO*28 cpr 15 mg  MYLAN PHARMACEUTICALS LIMITED  44285082   
800    ARIPIPRAZOLO*28 cpr 15 mg                    SANDOZ GMBH  44407310   
801  ARIPIPRAZOLO*28x1 cpr 15 mg          ACCORD HEALTHCARE SLU  44543142   

    Codice Gruppo Equivalenza  
797                       HSB  
798                       HSB  
799                       HSB  
800                       HSB 

In [409]:
df.isnull().sum()

,0
Principio Attivo,0
Descrizione Gruppo,0
Denominazione e Confezione,0
Titolare AIC,0
AIC,0
Codice Gruppo Equivalenza,0


In [410]:
df.duplicated().sum()

np.int64(0)

In [411]:
#!pip install pypdf
!pip install pymupdf

In [412]:
import requests
#from pypdf import PdfReader
import fitz  # PyMuPDF
import re

# Iterate over each row and get structured data from the API
for index, row in df.iterrows():

    aic = str(row['AIC']).zfill(9)

    # Build the API URL
    url = f"https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/formadosaggio/ricerca?query={aic}&spellingCorrection=true&page=0"

    # Make the GET request
    response = requests.get(url)

    # Check for successful response
    if response.status_code == 200:
        json_data = response.json()  # Parse the response as JSON

        # Access the nested data field
        content = json_data.get('data', {}).get('content', [])
        if len(content) == 0:
          continue

        # Print each item in the content list
        for item in content:
            id = item.get('id')
            codice_atc = item.get('codiceAtc')[0]
            df.at[index, 'ATC'] = codice_atc
            descrizione_atc = item.get('descrizioneAtc')
            codice_sis = item.get('medicinale').get('codiceSis')
            aic6 = item.get('medicinale').get('aic6')
            leaflet_url = f"https://medicinali.aifa.gov.it/it/#/it/organizzazione/{codice_sis}/farmaci/{aic6}/stampati/FI"
            pdf_url = f"https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/organizzazione/{codice_sis}/farmaci/{aic6}/stampati?ts=RCP"
            df.at[index, 'URL'] = leaflet_url
            df.at[index, 'PDF URL'] = pdf_url
            print(f"\n--- codiceAtc: {codice_atc} ---")
            print(f"\n--- descrizioneAtc: {descrizione_atc} ---")
            print(f"\n--- codiceSis: {codice_sis} ---")
            print(f"\n--- id: {id} ---")
            print(f"\n--- leafletUrl: {leaflet_url} ---")
            print(f"\n--- pdf_url: {pdf_url} ---")


            # Download the PDF
            response = requests.get(pdf_url)

            if response.status_code == 200:
                # Load and read the PDF
                pdf_path = 'rcp_downloaded.pdf'

                if response.status_code == 200:
                  with open("/content/rcp_downloaded.pdf", "wb") as f:
                      f.write(response.content)
                  print("PDF downloaded and saved as /content/rcp_downloaded.pdf")
                else:
                  print(f"Failed to download PDF. Status code: {response.status_code}")

                # creating a pdf reader object
                #doc = PdfReader(pdf_path)
                doc = fitz.open(pdf_path)

                # Combine all page text into one string
                full_text = ""
                #for page in doc.pages:
                for page in doc:
                    #full_text += page.extract_text().strip()
                    full_text += page.get_text()
                doc.close()

                # Normalize the text: fix words broken across lines
                text = re.sub(r'([a-zA-Z])\n([a-zA-Z])', r'\1 \2', full_text)  # fix broken words
                text = re.sub(r'\n+', '\n', text)
                text = re.sub(r'\s{2,}', ' ', text)  # collapse long spaces

                # Define section headings (number + title)
                sections = [
                    ("4.1", "Indicazioni terapeutiche"),
                    ("4.2", "Posologia e modo di somministrazione"),
                    ("4.3", "Controindicazioni"),
                    ("4.4", "Avvertenze speciali e precauzioni di impiego"),
                    ("4.5", "Interazioni con altri medicinali ed altre forme di interazione"),
                    ("4.6", "Fertilità, gravidanza e allattamento"),
                    ("4.7", "Effetti sulla capacità di guidare veicoli e sull’uso di macchinari"),
                    ("4.8", "Effetti indesiderati"),
                    ("4.9", "Sovradosaggio"),
                    ("6.2", "Incompatibilità")
                ]

                # Create a regex pattern for section headers
                section_regex = [
                    fr"({num}[\.\s]*{re.escape(title)})"
                    for num, title in sections
                ]
                pattern = '|'.join(section_regex)
                pattern = re.compile(pattern, flags=re.IGNORECASE)

                # Find all section starts
                matches = list(pattern.finditer(text))

                # Extract section contents
                extracted = {}
                for i in range(len(matches)):
                    start = matches[i].start()
                    end = matches[i+1].start() if i+1 < len(matches) else len(text)

                    header = matches[i].group().strip()
                    content = text[start:end].replace(header, '', 1).strip()

                    # Match section by header prefix (e.g. "4.2")
                    matched_section = next((f"{num} {title}" for num, title in sections if num in header), header)
                    extracted[matched_section] = content

                # Save each section in a separate variable
                for title, content in extracted.items():
                    section_name = title.replace(" ", "_").replace(".", "")  # Create a valid variable name
                    globals()[section_name] = content  # Dynamically assign the content to a variable
                    df.at[index, title] = content.replace(title," ").strip()

                    # Print the results
                    print(f"\n{'='*80}\n{title}\n{'='*80}\n{content}...\n")

    else:
        print(f"Request failed for AIC {aic}: Status Code {response.status_code}")

Streaming output truncated to the last 5000 lines.
Esula dalla competenza dell’AIFA ogni eventuale disputa concernente i diritti di proprietà industriale e la tutela brevettuale dei dati relativi all’AIC dei medicinali e, pertanto, l’Agenzia non può essere ritenuta responsabile in alcun modo di eventuali violazioni da parte del titolare dell'autorizzazione all'immissione in commercio (o titolare AIC). 8 giorno nelle donne in postmenopausa con carcinoma della mammella in fase avanzata; questi dosaggi sono stati ben tollerati. Non è stata stabilita la dose singola di anastrozolo in grado di provocare sintomi tali da porre il soggetto in pericolo di vita. Non esiste un antidoto specifico in caso di sovradosaggio ed il trattamento deve essere sintomatico. Nella gestione di un sovradosaggio occorre tenere in considerazione la possibilità che siano stati assunti diversi farmaci. Se il soggetto è vigile, può essere indotto il vomito. La dialisi può essere d’aiuto, in quanto l’anastrozolo non 

In [413]:
df = df.fillna("Not available")

In [414]:
df.head()

,Principio Attivo,Descrizione Gruppo,Denominazione e Confezione,Titolare AIC,AIC,Codice Gruppo Equivalenza,ATC,URL,PDF URL,4.1 Indicazioni terapeutiche,4.2 Posologia e modo di somministrazione,4.3 Controindicazioni,4.4 Avvertenze speciali e precauzioni di impiego,4.5 Interazioni con altri medicinali ed altre forme di interazione,"4.6 Fertilità, gravidanza e allattamento",4.7 Effetti sulla capacità di guidare veicoli e sull’uso di macchinari,4.8 Effetti indesiderati,4.9 Sovradosaggio,6.2 Incompatibilità
470,Amitriptilina,AMITRIPTILINA 25MG 25 UNITA' USO ORALE,TRIPTIZOL*25 cpr riv 25 mg,SIT LABORATORIO FARMAC. Srl,19803016,HOB,N06AA09,https://medicinali.aifa.gov.it/it/#/it/organiz...,https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/...,Triptizol è indicato per: • il trattamento del...,Posologia Agenzia Italiana del Farmaco Documen..., Ipersensibilità al principio attivo o ad uno...,Agenzia Italiana del Farmaco Documento reso di...,Potenziali effetti di amitriptilina su altri m...,Gravidanza Per amitriptilina sono disponibili ...,Amitriptilina è un medicinale sedativo. I pazi...,Amitriptilina può indurre effetti indesiderati...,"Sintomi Sintomi anticolinergici: midriasi, tac...",Non pertinente. 6.3 Periodo di validità 5 anni...
471,Amitriptilina,AMITRIPTILINA 25MG 30 UNITA' USO ORALE,ADEPRIL*30 cpr riv 25 mg,TEOFARMA Srl,20019028,HOC,N06AA09,https://medicinali.aifa.gov.it/it/#/it/organiz...,https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/...,Adepril è indicato per:  il trattamento del d...,Posologia Agenzia Italiana del Farmaco Documen...,Ipersensibilità al principio attivo o ad uno q...,Not available,Potenziali effetti di amitriptilina su altri m...,Gravidanza Per amitriptilina sono disponibili ...,Amitriptilina è un medicinale sedativo. Agenzi...,Amitriptilina può indurre effetti indesiderati...,"Sintomi Sintomi anticolinerigici: midriasi, ta...",L’associazione con altri psicofarmaci richiede...
472,Amitriptilina,AMITRIPTILINA 40MG/ML 20ML USO ORALE,LAROXYL*orale gtt 20 ml 40 mg/ml,TEOFARMA Srl,19906054,HOD,N06AA09,https://medicinali.aifa.gov.it/it/#/it/organiz...,https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/...,Laroxyl è indicato per:  il trattamento del d...,Agenzia Italiana del Farmaco Documento reso di...,Ipersensibilità al principio attivo o ad uno q...,Con dosi elevate è probabile la comparsa di ar...,Potenziali effetti di amitriptilina su altri m...,Gravidanza Per amitriptilina sono disponibili ...,Not available,Amitriptilina può indurre effetti indesiderati...,"Sintomi Sintomi anticolinerigici: midriasi, ta...",Non pertinente. 6.3 Periodo di validità compre...
473,Amitriptilina,AMITRIPTILINA 40MG/ML 20ML USO ORALE,AMITRIPTILINA*orale gtt 20 ml 40 mg/ml,EG SpA,44532012,HOD,N06AA09,https://medicinali.aifa.gov.it/it/#/it/organiz...,https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/...,Amitriptilina EG è indicato per:  il trattame...,Posologia Disturbo depressivo maggiore Iniziar...,Ipersensibilità al principio attivo o ad uno q...,Con dosi elevate è probabile la comparsa di ar...,Not available,Gravidanza Per amitriptilina sono disponibili ...,Amitriptilina è un medicinale sedativo. I pazi...,Amitriptilina può indurre effetti indesiderati...,"Sintomi Sintomi anticolinerigici: midriasi, ta...",Non pertinente. 6.3. Periodo di validità 3 ann...
474,Amitriptilina,AMITRIPTILINA 40MG/ML 20ML USO ORALE,AMITRIPTILINA*orale gtt 20 ml 40 mg/ml,DOC GENERICI Srl,45218017,HOD,N06AA09,https://medicinali.aifa.gov.it/it/#/it/organiz...,https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/...,AMITRIPTILINA DOC è indicato per - il trattame...,Se il dosaggio prescritto non fosse raggiungib...,- Ipersensibilità al principio attivo o ad uno...,Con dosi elevate è probabile la comparsa di ar...,Potenziali effetti di amitriptilina su altri m...,Gravidanza Per amitriptilina sono disponibili ...,Amitriptilina è un medicinale sedativo. I pazi...,Amitriptilina può indurre effetti indesiderati...,"Sintomi Sintomi anticolinerigici: midriasi, ta...",Non pertinente.

In [415]:
df.tail()

,Principio Attivo,Descrizione Gruppo,Denominazione e Confezione,Titolare AIC,AIC,Codice Gruppo Equivalenza,ATC,URL,PDF URL,4.1 Indicazioni terapeutiche,4.2 Posologia e modo di somministrazione,4.3 Controindicazioni,4.4 Avvertenze speciali e precauzioni di impiego,4.5 Interazioni con altri medicinali ed altre forme di interazione,"4.6 Fertilità, gravidanza e allattamento",4.7 Effetti sulla capacità di guidare veicoli e sull’uso di macchinari,4.8 Effetti indesiderati,4.9 Sovradosaggio,6.2 Incompatibilità
797,Aripiprazolo,ARIPIPRAZOLO 15MG 28 UNITA' USO ORALE,ARIPIPRAZOLO*28 cpr 15 mg,EG SpA,43801101,HSB,N05AX12,https://medicinali.aifa.gov.it/it/#/it/organiz...,https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/...,L’aripiprazolo è indicato per il trattamento d...,Posologia Adulti Schizofrenia: la dose di part...,Ipersensibilità al principio attivo o a uno qu...,Not available,Not available,Gravidanza Non ci sono studi specifici e adegu...,"Come con altri antipsicotici, i pazienti devon...",Sintesi del profilo di sicurezza Le reazioni a...,Segni e sintomi Negli studi clinici e nell'esp...,Non pertinente. Agenzia Italiana del Farmaco D...
798,Aripiprazolo,ARIPIPRAZOLO 15MG 28 UNITA' USO ORALE,ARIPIPRAZOLO*28 cpr 15 mg,ZENTIVA K.S.,44265128,HSB,N05AX12,https://medicinali.aifa.gov.it/it/#/it/organiz...,https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/...,Aripiprazolo Zentiva è indicato per il trattam...,Posologia Adulti Schizofrenia La dose di parte...,Ipersensibilità al principio attivo o ad uno q...,Not available,Not available,Not available,Aripiprazolo altera lievemente o moderatamente...,Riassunto del profilo di sicurezza Le reazioni...,Segni e sintomi Negli studi clinici e successi...,Non pertinente. 6.3 Periodo di validità 2 anni...
799,Aripiprazolo,ARIPIPRAZOLO 15MG 28 UNITA' USO ORALE,ARIPIPRAZOLO*28 cpr 15 mg,MYLAN PHARMACEUTICALS LIMITED,44285082,HSB,N05AX12,https://medicinali.aifa.gov.it/it/#/it/organiz...,https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/...,Aripiprazolo Mylan Pharma è indicato per il tr...,Posologia Adulti Schizofrenia: la dose di part...,Ipersensibilità al principio attivo o ad uno q...,Not available,Not available,Gravidanza Non ci sono studi adeguati e valida...,Aripiprazolo altera lievemente o moderatamente...,Riassunto del profilo di sicurezza Le reazioni...,Segni e sintomi Negli studi clinici e nell’esp...,Non pertinente. 6.3 Periodo di validità 3 anni...
800,Aripiprazolo,ARIPIPRAZOLO 15MG 28 UNITA' USO ORALE,ARIPIPRAZOLO*28 cpr 15 mg,SANDOZ GMBH,44407310,HSB,N05AX12,https://medicinali.aifa.gov.it/it/#/it/organiz...,https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/...,Aripiprazolo Sandoz è indicato per il trattame...,Posologia Adulti Schizofrenia: la dose di part...,Ipersensibilità al principio attivo o ad uno q...,Not available,Not available,Not available,Aripiprazolo altera lievemente o moderatamente...,Riassunto del profilo di sicurezza Le reazioni...,Segni e sintomi Negli studi clinici e nell’esp...,Non pertinente. 6.3 Periodo di validità 2 anni...
801,Aripiprazolo,ARIPIPRAZOLO 15MG 28 UNITA' USO ORALE,ARIPIPRAZOLO*28x1 cpr 15 mg,ACCORD HEALTHCARE SLU,44543142,HSB,N05AX12,https://medicinali.aifa.gov.it/it/#/it/organiz...,https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/...,Aripiprazolo Accord è indicato per il trattame...,Posologia Adulti Schizofrenia: la dose di part...,Ipersensibilità al principio attivo o ad uno q...,Not available,Not available,Not available,Aripiprazolo altera lievemente o moderatamente...,Riassunto del profilo di sicurezza Le reazioni...,Segni e sintomi Negli studi clinici e nell'esp...,Non pertinente. 6.3 Periodo di validità 2 anni...


In [416]:
df.to_csv('updated_data.csv', index=False)
print("\nUpdated CSV saved as updated_data.csv")


Updated CSV saved as updated_data.csv
